In [3]:
from datetime import datetime
from pathlib import Path
import gsw
import numpy as np
import pandas as pd
import xarray as xr
import pyarrow.parquet as pq
import requests as rq
import os
from urllib.parse import urljoin, urlencode
import argo_tools as at
import argopy
from argopy import DataFetcher as ArgoDataFetcher
local_gdac = '/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData'
#Path.mkdir('/vortexfs1/share/boom/projects/n2o/pq')
outdir = '/vortexfs1/share/boom/projects/n2o/pq'
argopy.set_options(mode='expert',src='gdac',ftp=local_gdac)

In [5]:
wmos,df2,wmo_fp = at.argo_gdac()

FileNotFoundError: [Errno 2] No such file or directory: './argo_synthetic-profile_index.txt'

In [6]:
import sys
from itertools import islice

if sys.version_info >= (3, 12):
    from itertools import batched
else:
    try:
        from more_itertools import batched
    except ImportError:
        def batched(iterable, chunk_size):
            iterator = iter(iterable)
            while chunk := tuple(islice(iterator, chunk_size)):
                yield chunk

In [7]:
VARS = ['JULD','CYCLE_NUMBER','PLATFORM_NUMBER','LATITUDE','LONGITUDE',
 'PRES',
 'PRES_QC',
 'PRES_ADJUSTED',
 'PRES_ADJUSTED_QC',
 'PRES_ADJUSTED_ERROR',
 'TEMP',
 'TEMP_QC',
 'TEMP_dPRES',
 'TEMP_ADJUSTED',
 'TEMP_ADJUSTED_QC',
 'TEMP_ADJUSTED_ERROR',
 'PSAL',
 'PSAL_QC',
 'PSAL_dPRES',
 'PSAL_ADJUSTED',
 'PSAL_ADJUSTED_QC',
 'PSAL_ADJUSTED_ERROR',
 'DOXY',
 'DOXY_QC',
 'DOXY_dPRES',
 'DOXY_ADJUSTED',
 'DOXY_ADJUSTED_QC',
 'DOXY_ADJUSTED_ERROR',
 'CHLA',
 'CHLA_QC',
 'CHLA_dPRES',
 'CHLA_ADJUSTED',
 'CHLA_ADJUSTED_QC',
 'CHLA_ADJUSTED_ERROR',
 'BBP700',
 'BBP700_QC',
 'BBP700_dPRES',
 'BBP700_ADJUSTED',
 'BBP700_ADJUSTED_QC',
 'BBP700_ADJUSTED_ERROR',
 'CDOM',
 'CDOM_QC',
 'CDOM_dPRES',
 'CDOM_ADJUSTED',
 'CDOM_ADJUSTED_QC',
 'CDOM_ADJUSTED_ERROR',
 'PH_IN_SITU_TOTAL',
 'PH_IN_SITU_TOTAL_QC',
 'PH_IN_SITU_TOTAL_dPRES',
 'PH_IN_SITU_TOTAL_ADJUSTED',
 'PH_IN_SITU_TOTAL_ADJUSTED_QC',
 'PH_IN_SITU_TOTAL_ADJUSTED_ERROR',
 'NITRATE',
 'NITRATE_QC',
 'NITRATE_dPRES',
 'NITRATE_ADJUSTED',
 'NITRATE_ADJUSTED_QC',
 'NITRATE_ADJUSTED_ERROR']

### Parallel 

In [ ]:
import multiprocessing

CHUNK_SZ = 100
NPROC = 12
#VARS = ['LATITUDE','LONGITUDE','PRES','PRES_ADJUSTED','DOXY_ADJUSTED','DOXY_ADJUSTED_QC','PSAL','TEMP','CYCLE_NUMBER','PLATFORM_NUMBER']

chunks = batched(wmo_fp,CHUNK_SZ)


def chunk2pq(chunk):
    dflist = []
    for line in chunk:  
        if '6902958' in line:
            print('skipping 6902958')
        else:
            fn = (line.split('/')[1] + "_Sprof.nc")
            fpath = Path(local_gdac) / "dac" / line / fn
            try:
                ds = xr.load_dataset(fpath,engine="argo") # loading single profile dataset
            except:
                print('Failed on ' + str(fpath))             
            invars = list(set(VARS) & set(list(ds.data_vars)))
            df = ds[invars].to_dataframe()
            dflist.append(df)
    print(fpath)
    df = pd.concat(dflist)
    #df['PLATFORM_NUMBER'] = df['PLATFORM_NUMBER'].str.strip()
    df.to_parquet(Path(outdir) / ("test" + line.split('/')[1] + ".parquet"),coerce_timestamps='us',allow_truncated_timestamps=True)
    
pool_obj = multiprocessing.Pool(processes=NPROC)
pool_obj.map(chunk2pq,chunks)
pool_obj.close()


### Test direct subsetting from parquet directory

In [ ]:
#t = pq.read_table(Path(outdir),filters=[('PRES', '<', 20),('LATITUDE', '<', 21.1),('LATITUDE', '>', 21)])
t = pq.read_table(Path(outdir),filters=[('PLATFORM_NUMBER', '==', 1902304)])
df = t.to_pandas()
df
#ds = xr.Dataset.from_dataframe(df)
#ds

### Example loading Sprof from snapshot
```
ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc')
df = ds.to_dataframe()
```

In [ ]:

from dotenv import load_dotenv
load_dotenv()
import os
from pyarrow import fs
s3 = fs.S3FileSystem(region='us-east-1')


In [ ]:
s3

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import Table

ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc',engine="argo")
df = ds[['DOXY','PRES','NITRATE','PLATFORM_NUMBER']].to_dataframe()

s3_filepath = 'data.parquet'

pq.write_to_dataset(
    Table.from_pandas(df),
    s3_filepath,
    filesystem=s3,
    use_dictionary=True,
    compression="snappy",
    version="2.4",
)



### single threaded

In [ ]:

CHUNK_SZ = 100
VARS = ['JULD','LATITUDE','LONGITUDE','PRES','PRES_ADJUSTED','DOXY_ADJUSTED','DOXY_ADJUSTED_QC','NITRATE','NITRATE_ADJUSTED','PSAL','TEMP','CYCLE_NUMBER','PLATFORM_NUMBER']
for chunk in batched(wmo_fp,CHUNK_SZ):
    dflist = []
    for line in chunk:  
        fn = (line.split('/')[1] + "_Sprof.nc")
        fpath = Path(local_gdac) / "dac" / line / fn
        try:
            ds = xr.load_dataset(fpath)
        except:
            print(fpath)
        invars = list(set(VARS) & set(list(ds.data_vars)))
        df = ds[invars].to_dataframe()
        dflist.append(df)
    print(fpath)
    df = pd.concat(dflist)
    df.to_parquet('pq/test' + line.split('/')[1] + ".parquet",coerce_timestamps='us',allow_truncated_timestamps=True)
        
        